In [2]:
import pandas as pd
import numpy as np
import json

Figure out how to get gameids that we'll loop through later

In [2]:
with open("../pfr_pages/team_season_pages_2017.json",'r') as f:
    season_pages = json.load(f)

In [3]:
[x for x in season_pages.keys()][:4]

['nwe2017', 'buf2017', 'mia2017', 'nyj2017']

In [4]:
[x for x in season_pages['nwe2017']][:]

['team_stats_clone',
 'team_stats',
 'games_clone',
 'games',
 'team_conversions',
 'passing_clone',
 'passing',
 'rushing_and_receiving_clone',
 'rushing_and_receiving',
 'returns',
 'kicking_clone',
 'kicking',
 'defense',
 'scoring_clone',
 'scoring',
 'team_td_log',
 'opp_td_log']

In [5]:
pd.DataFrame(season_pages['nwe2017']['games']).columns

Index(['boxscore_word', 'boxscore_word_a', 'boxscore_word_href', 'exp_pts_def',
       'exp_pts_off', 'exp_pts_st', 'first_down_def', 'first_down_off',
       'game_date', 'game_day_of_week', 'game_location', 'game_outcome',
       'game_time', 'opp', 'opp_a', 'opp_href', 'overtime', 'pass_yds_def',
       'pass_yds_off', 'pts_def', 'pts_off', 'rush_yds_def', 'rush_yds_off',
       'team_record', 'to_def', 'to_off', 'week_num', 'yards_def',
       'yards_off'],
      dtype='object')

In [6]:
[x for x in season_pages['nwe2017']['games']]

[{'boxscore_word_a': 'boxscore',
  'boxscore_word_href': '/boxscores/201709070nwe.htm',
  'exp_pts_def': '-23.18',
  'exp_pts_off': '5.92',
  'exp_pts_st': '0.73',
  'first_down_def': '26',
  'first_down_off': '25',
  'game_date': 'September 7',
  'game_day_of_week': 'Thu',
  'game_location': '',
  'game_outcome': 'L',
  'game_time': '8:30PM ET',
  'opp_a': 'Kansas City Chiefs',
  'opp_href': '/teams/kan/2017.htm',
  'overtime': '',
  'pass_yds_def': '352',
  'pass_yds_off': '247',
  'pts_def': '42',
  'pts_off': '27',
  'rush_yds_def': '185',
  'rush_yds_off': '124',
  'team_record': '0-1',
  'to_def': '1',
  'to_off': '',
  'week_num': '1',
  'yards_def': '537',
  'yards_off': '371'},
 {'boxscore_word_a': 'boxscore',
  'boxscore_word_href': '/boxscores/201709170nor.htm',
  'exp_pts_def': '-12.16',
  'exp_pts_off': '27.58',
  'exp_pts_st': '-2.32',
  'first_down_def': '20',
  'first_down_off': '29',
  'game_date': 'September 17',
  'game_day_of_week': 'Sun',
  'game_location': '@',
  

In [7]:
cols = ['week_num', 'boxscore_word_href', 'opp_href', 'game_location']
pd.DataFrame(season_pages['nwe2017']['games'])[cols]

,week_num,boxscore_word_href,opp_href,game_location
0,1,/boxscores/201709070nwe.htm,/teams/kan/2017.htm,
1,2,/boxscores/201709170nor.htm,/teams/nor/2017.htm,@
2,3,/boxscores/201709240nwe.htm,/teams/htx/2017.htm,
3,4,/boxscores/201710010nwe.htm,/teams/car/2017.htm,
4,5,/boxscores/201710050tam.htm,/teams/tam/2017.htm,@
5,6,/boxscores/201710150nyj.htm,/teams/nyj/2017.htm,@
6,7,/boxscores/201710220nwe.htm,/teams/atl/2017.htm,
7,8,/boxscores/201710290nwe.htm,/teams/sdg/2017.htm,
8,9,NaN,NaN,
9,10,/boxscores/201711120den.htm,/teams/den/2017.htm,@


In [8]:
def get_games(season_pages,teamkey):
    games_info = []
    team_df = pd.DataFrame(season_pages[teamkey]['games'])
    for wk, gid, opp, loc in zip(team_df['week_num'],
                                 team_df['boxscore_word_href'],
                                 team_df['opp_href'],
                                 team_df['game_location']):
#        print(wk,gid,opp,loc)
        try:
            gid = str(gid).split("/")[2].split(".")[0]
            opp = str(opp).split("/")[2]
        except:
            gid = 'nan'
            opp = 'nan'
        
        yr = teamkey[3:]
        # Get home/away teams
        tm = teamkey[:3]
        if loc == "@":
            home = opp
            away = tm
        else:
            home = tm
            away = opp
    
#        print(wk,gid,tm,opp,loc,home,away)
        games_info.append((yr,wk,gid,loc,home,away))
        
    return games_info

get_games(season_pages,'nwe2017')

[('2017', '1', '201709070nwe', '', 'nwe', 'kan'),
 ('2017', '2', '201709170nor', '@', 'nor', 'nwe'),
 ('2017', '3', '201709240nwe', '', 'nwe', 'htx'),
 ('2017', '4', '201710010nwe', '', 'nwe', 'car'),
 ('2017', '5', '201710050tam', '@', 'tam', 'nwe'),
 ('2017', '6', '201710150nyj', '@', 'nyj', 'nwe'),
 ('2017', '7', '201710220nwe', '', 'nwe', 'atl'),
 ('2017', '8', '201710290nwe', '', 'nwe', 'sdg'),
 ('2017', '9', 'nan', '', 'nwe', 'nan'),
 ('2017', '10', '201711120den', '@', 'den', 'nwe'),
 ('2017', '11', '201711190rai', '@', 'rai', 'nwe'),
 ('2017', '12', '201711260nwe', '', 'nwe', 'mia'),
 ('2017', '13', '201712030buf', '@', 'buf', 'nwe'),
 ('2017', '14', '201712110mia', '@', 'mia', 'nwe'),
 ('2017', '15', '201712170pit', '@', 'pit', 'nwe'),
 ('2017', '16', '201712240nwe', '', 'nwe', 'buf'),
 ('2017', '17', '201712310nwe', '', 'nwe', 'nyj'),
 ('2017', '', 'nan', '', 'nwe', 'nan'),
 ('2017', 'Division', '201801130nwe', '', 'nwe', 'oti'),
 ('2017', 'Conf. Champ.', '201801210nwe', '', 

In [9]:
# Get all unique gameids for a season
l = [get_games(season_pages,x) for x in season_pages.keys()]
gameids = set([item[2] for sublist in l for item in sublist])
set([item[2] for sublist in l for item in sublist])

{'201709070nwe',
 '201709100buf',
 '201709100chi',
 '201709100cin',
 '201709100cle',
 '201709100dal',
 '201709100det',
 '201709100gnb',
 '201709100htx',
 '201709100oti',
 '201709100ram',
 '201709100sfo',
 '201709100was',
 '201709110den',
 '201709110min',
 '201709140cin',
 '201709170atl',
 '201709170car',
 '201709170clt',
 '201709170den',
 '201709170jax',
 '201709170kan',
 '201709170nor',
 '201709170pit',
 '201709170rai',
 '201709170ram',
 '201709170rav',
 '201709170sdg',
 '201709170sea',
 '201709170tam',
 '201709180nyg',
 '201709210sfo',
 '201709240buf',
 '201709240car',
 '201709240chi',
 '201709240clt',
 '201709240det',
 '201709240gnb',
 '201709240jax',
 '201709240min',
 '201709240nwe',
 '201709240nyj',
 '201709240oti',
 '201709240phi',
 '201709240sdg',
 '201709240was',
 '201709250crd',
 '201709280gnb',
 '201710010atl',
 '201710010cle',
 '201710010crd',
 '201710010dal',
 '201710010den',
 '201710010htx',
 '201710010mia',
 '201710010min',
 '201710010nwe',
 '201710010nyj',
 '201710010rav

## Figure out how to pull individual game play-by-play

In [10]:
with open("../pfr_pages/game_pages.json",'r') as f:
    game_pages = json.load(f)

What do individual game keys look like?

In [11]:
[x for x in game_pages.keys()][:5]

['201709070nwe',
 '201709170nor',
 '201709240nwe',
 '201710010nwe',
 '201710050tam']

What tables are available for any given game?

In [12]:
[x for x in game_pages['201709070nwe'].keys()]

['scoring',
 'game_info',
 'officials',
 'expected_points',
 'team_stats',
 'player_offense',
 'player_defense',
 'returns',
 'kicking',
 'home_starters',
 'vis_starters',
 'home_snap_counts',
 'vis_snap_counts',
 'targets_directions',
 'rush_directions',
 'pass_tackles',
 'rush_tackles',
 'home_drives',
 'vis_drives',
 'pbp_clone',
 'pbp']

Make a DataFrame from one of these tables

In [13]:
pd.DataFrame(game_pages['201709070nwe']['game_info'])

,info,stat
0,Won Toss,Chiefs (deferred)
1,Roof,outdoors
2,Surface,fieldturf
3,Weather,"63 degrees, wind 8 mph"
4,Vegas Line,New England Patriots -8.0
5,Over/Under,47.5


In [14]:
pd.DataFrame(game_pages['201709070nwe']['home_drives'])

,drive_num,end_event,net_yds,play_count_tip,quarter,rowclass,start_at,time_start,time_total
0,1,Touchdown,73,,1,bold,NWE 27,15:00,2:52
1,2,Downs,22,,1,NaN,KAN 32,12:00,2:35
2,3,Field Goal,67,,1,bold,NWE 25,03:03,3:07
3,4,Touchdown,82,,2,bold,NWE 18,13:17,5:50
4,5,Punt,4,,2,NaN,NWE 40,03:44,0:57
5,6,End of Half,-1,,2,NaN,NWE 18,00:13,0:13
6,7,Punt,9,,3,NaN,NWE 17,11:53,2:25
7,8,Punt,26,,3,NaN,NWE 25,09:19,2:11
8,9,Touchdown,54,,3,bold,NWE 46,06:22,1:21
9,10,Field Goal,58,,3,bold,NWE 28,02:50,2:41


In [15]:
off_df = pd.DataFrame(game_pages['201709070nwe']['player_offense'])
def_df = pd.DataFrame(game_pages['201709070nwe']['player_defense'])
kick_df = pd.DataFrame(game_pages['201709070nwe']['kicking'])
plyrs_df = pd.concat([off_df[['team','player_href']],
                      def_df[['team','player_href']],
                      kick_df[['team','player_href']] ])
home_bool = [True if str(x).lower()=='nwe' else False for x in plyrs_df.team.values]
away_bool = [True if str(x).lower()=='kan' else False for x in plyrs_df.team.values]
home_plyrs = plyrs_df[home_bool].player_href.values
away_plyrs = plyrs_df[away_bool].player_href.values
away_plyrs

array(['/players/S/SmitAl03.htm', '/players/H/HuntKa00.htm',
       '/players/W/WestCh01.htm', '/players/H/HillTy00.htm',
       '/players/K/KelcTr00.htm', '/players/T/ThomDe05.htm',
       '/players/C/ConlCh00.htm', '/players/W/WilsAl02.htm',
       '/players/H/HarrDe03.htm', '/players/T/TravRo00.htm',
       '/players/B/BailAl00.htm', '/players/B/BerrEr99.htm',
       '/players/F/FordDe00.htm', '/players/G/GainPh00.htm',
       '/players/H/HillTy00.htm', '/players/H/HousJu00.htm',
       '/players/J/JohnDe25.htm', '/players/L/LogaBe00.htm',
       '/players/M/MillRo99.htm', '/players/M/MitcTe00.htm',
       '/players/M/MurrEr00.htm', '/players/P/ParkRo01.htm',
       '/players/P/PeteMa00.htm', '/players/S/SoreDa01.htm',
       '/players/Z/ZombFr00.htm', '/players/S/SantCa01.htm',
       '/players/C/ColqDu20.htm'], dtype=object)

In [16]:
pd.DataFrame(game_pages['201709070nwe']['pbp'])

,detail,detail_a,detail_href,detail_text,down,exp_pts_after,exp_pts_before,home_wp,location,onecell,pbp_score_aw,pbp_score_hm,qtr_time_remain,qtr_time_remain_a,qtr_time_remain_href,quarter,rowclass,yds_to_go
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1st Quarter,NaN,NaN,NaN,NaN,NaN,NaN,thead onecell,NaN
1,NaN,"Cairo Santos, Dion Lewis, Kevin Pierre-Louis","/players/S/SantCa01.htm, /players/L/LewiDi00.h...","kicks off 64 yards, returned by brk, for 26 ...",,-0.740,0.000,74.2,KAN 35,NaN,0,0,NaN,15:00,#pbp_1,1,NaN,
2,NaN,"Tom Brady, Dwayne Allen","/players/B/BradTo00.htm, /players/A/AlleDw00.htm",pass incomplete deep left intended for,1,0.200,0.740,72.9,NWE 27,NaN,0,0,NaN,14:55,#pbp_2,1,divider,10
3,NaN,"Tom Brady, Rex Burkhead, Ron Parker","/players/B/BradTo00.htm, /players/B/BurkRe00.h...","pass complete short right to brk, for 8 yard...",2,0.560,0.200,73.7,NWE 27,NaN,0,0,NaN,14:49,#pbp_3,1,NaN,10
4,NaN,"James White, Ron Parker, Derrick Johnson","/players/W/WhitJa02.htm, /players/P/ParkRo01.h...","left guard for 8 yards (tackle by brk, and b...",3,1.800,0.560,76.6,NWE 35,NaN,0,0,NaN,14:14,#pbp_4,1,NaN,2
5,NaN,"James White, Eric Berry","/players/W/WhitJa02.htm, /players/B/BerrEr99.htm","middle for 3 yards (tackle by brk, )",1,1.660,1.800,76.2,NWE 43,NaN,0,0,NaN,13:52,#pbp_5,1,NaN,10
6,NaN,"Tom Brady, Brandin Cooks, Phillip Gaines","/players/B/BradTo00.htm, /players/C/CookBr00.h...","pass complete deep left to brk, for 19 yards...",2,3.250,1.660,79.7,NWE 46,NaN,0,0,NaN,13:26,#pbp_6,1,NaN,7
7,NaN,"James White, Dee Ford, Derrick Johnson","/players/W/WhitJa02.htm, /players/F/FordDe00.h...","left tackle for 5 yards (tackle by brk, and ...",1,3.380,3.250,80.0,KAN 35,NaN,0,0,NaN,13:02,#pbp_7,1,NaN,10
8,NaN,"Tom Brady, Danny Amendola, Phillip Gaines","/players/B/BradTo00.htm, /players/A/AmenDa00.h...","pass complete short left to brk, for 16 yard...",2,4.650,3.380,82.5,KAN 30,NaN,0,0,NaN,12:35,#pbp_8,1,NaN,5
9,NaN,"Tom Brady, Brandin Cooks, Terrance Mitchell","/players/B/BradTo00.htm, /players/C/CookBr00.h...","pass incomplete short left intended for brk, ...",1,6.740,4.650,86.3,KAN 14,NaN,0,0,NaN,12:24,#pbp_9,1,NaN,10


In [17]:
# Make a dictionary to store what three-letter code each 
# team used in a given season
three_letter_code = {}
# Most teams don't change
tms = [
    'atl','nor','car','tam','nyg','dal','phi','was',
    'min','gnb','det','chi','sea','sfo','crd',
    'pit','cle','cin','buf','nyj','mia','nwe',
    'jax','htx','den','kan'
]
for tm in tms:
    for yr in range(2000,2019):
        k = '{}{}'.format(tm,yr)
        three_letter_code[k] = tm
# Arizona Cardinals
for yr in range(2000,2019):
    k = 'crd{}'.format(yr)
    three_letter_code[k] = 'ari'
# St Louis Rams
    # 2016-17 - lar
    # 2000-15 - stl
three_letter_code['ram2017'] = 'lar'
three_letter_code['ram2016'] = 'lar'
for yr in range(2000,2016):
    k = 'ram{}'.format(yr)
    three_letter_code[k] = 'stl'
# Baltimore Ravens
for yr in range(2000,2019):
    k = 'rav{}'.format(yr)
    three_letter_code[k] = 'bal'
# Oilers/Titans
for yr in range(2000,2019):
    k = 'oti{}'.format(yr)
    three_letter_code[k] = 'ten'
# Indianapolis Colts
for yr in range(2000,2019):
    k = 'clt{}'.format(yr)
    three_letter_code[k] = 'ind'
# Texans
for yr in range(2000,2019):
    k = 'htx{}'.format(yr)
    three_letter_code[k] = 'hou'
# Raiders
for yr in range(2000,2019):
    k = 'rai{}'.format(yr)
    three_letter_code[k] = 'oak'
# Chargers
three_letter_code['sdg2017'] = 'lac'
for yr in range(2000,2018):
    k = 'sdg{}'.format(yr)
    three_letter_code[k] = 'sdg'

In [18]:
def get_secs_rem(df):
    # Get column for seconds remaining
    qtr = df['quarter'].values
    time_rem = df['qtr_time_remain_a'].values
    secs_rem = []
    
    for q, tr in zip(qtr, time_rem):
        try:
            [mins, secs] = tr.split(":")
            sr = 900*(4-int(q)) + 60*int(mins) + int(secs)
            secs_rem.append(sr)
        except:
            if 3600 in secs_rem:
                secs_rem.append(secs_rem[-1])
            else:
                secs_rem.append(3600)
            
    return secs_rem
    

def get_possession(df, gameinfo):
    (ssn, wk, gid, loc, home, away) = gameinfo
    
    # Get lists of players for home and away teams
    # First try and grab by snap counts
    if 'home_snap_counts' in game_pages[gid].keys():
        home_snaps = pd.DataFrame(game_pages[gid]['home_snap_counts'])
        home_plyrs = home_snaps.player_href.values
        away_snaps = pd.DataFrame(game_pages[gid]['vis_snap_counts'])
        away_plyrs = away_snaps.player_href.values
    # Otherwise try getting players from individual stats tables
    elif 'player_offense' in game_pages[gid].keys():
        off_df = pd.DataFrame(game_pages[gid]['player_offense'])
        def_df = pd.DataFrame(game_pages[gid]['player_defense'])
        kick_df = pd.DataFrame(game_pages[gid]['kicking'])
        plyrs_df = pd.concat([off_df[['team','player_href']],
                              def_df[['team','player_href']],
                              kick_df[['team','player_href']] ])
        hm_tm = three_letter_code[home+str(ssn)]
        aw_tm = three_letter_code[away+str(ssn)]
        home_bool = [True if str(x).lower()==hm_tm else False for x in plyrs_df.team.values]
        away_bool = [True if str(x).lower()==aw_tm else False for x in plyrs_df.team.values]
        home_plyrs = plyrs_df[home_bool].player_href.values
        away_plyrs = plyrs_df[away_bool].player_href.values
        
    # Make sure everything is lowercase
    home_plyrs = [str(x).lower() for x in home_plyrs]
    away_plyrs = [str(x).lower() for x in away_plyrs]
    
    # Go through and track possession changes.
    poss = []
    for i, (rc, plyrs) in enumerate(zip(df['rowclass'].values,
                                        df['detail_href'].values)):
#        print("")
#        print("rowclass",rc)
#        print("detail_href",str(plyrs))
        # If rowclass is divider or score (or previous row is onecell), 
        # ball is probably controlled by team of first player mentioned
        if poss == []:
#            print("Adding first entry into possession")
            poss.append(home)
        elif str(plyrs) == 'nan':
#            print("No players in this play, using previous play's possession")
            poss.append(poss[-1])
        else:
            prev_rc = df.rowclass.values[i-1]
#            print("prev rowclass =",prev_rc)
            plyr1 = plyrs.split(", ")[0].lower()
#            print("maybe looking to match",plyr1)
            if ("divider" in str(rc)) or ("score" in str(rc)):
                if plyr1 in home_plyrs:
                    poss.append(home)
                elif plyr1 in away_plyrs:
                    poss.append(away)
                else:
                    print("Couldn't match player",plyr1)
                    poss.append(poss[-1])
            elif "onecell" in str(prev_rc):
                if plyr1 in home_plyrs:
                    poss.append(home)
                elif plyr1 in away_plyrs:
                    poss.append(away)
                else:
                    print("Couldn't match player",plyr1)
                    poss.append(poss[-1])
            else:
#                print("Continuing possession from previous play")
                poss.append(poss[-1])
            
    return poss
    
    
def get_fieldposition(df, gameinfo):
    """
    Write fieldposition from the offense's perspective
    """
    (ssn, wk, gid, loc, home, away) = gameinfo
    off_fp = []
    for off, loc in zip(df.offense.values, df.location.values):
#        print(off, loc)
        if '50' in str(loc):
            off_fp.append(0)
        elif (str(loc) == '') or (str(loc) == 'nan') or ("Location" in str(loc)):
            off_fp.append(0)
        elif len(loc) >= 4:
            side = loc.split()[0].lower()
            ydline = int(loc.split()[1])
            if side == off:
                # Offense is on own side of field. fieldpos is negative.
                off_fp.append( -1*(50-ydline) )
            else:
                # Offense is in opposing territory
                off_fp.append( 50-ydline )
        else:
            print("not sure what to do with",loc)
            off_fp.append(0)
            
    return off_fp


def found_pass(detail):
    d = detail.lower()
    pass_terms = ["pass", "sacked", "scramble",
                  "interception", "intercepted"]
    for term in pass_terms:
        if term in d:
            return True
    return False

def found_scramble(detail):
    d = detail.lower()
    if "scramble" in d:
        return True
    return False
    
def found_run(detail):
    d = detail.lower()
    run_terms = ["run ", "rush", "left tackle ", "up the middle ",
                 "left end ", "right end ", "left guard ", "right guard "]
    if not "scramble" in d:
        for term in run_terms:
            if term in d:
                return True
    return False

        
def found_punt(detail):
    d = detail.lower()
    if " punts " in d:
        return True
    elif " punt return" in d:
        return True
    return False
    
def found_fieldgoal(detail):
    d = detail.lower()
    if " field goal" in d:
        return True
    return False

def yds_run( detail ):
    words = detail.lower().split()
    # look for yardage in format "for X yards"
    for j, w in enumerate(words):
        if w == "for" and len(words) > j+2:
            if words[j+2].rstrip(".,") in ("yd","yds","yrd","yrds","yard","yards"):
                return int(words[j+1])
            # or "for no gain"
            elif "no" in words[j+1] and "gain" in words[j+2]:
                return 0
        
        # or "X yard run/rush"
        elif w in ("yd","yds","yrd","yrds","yard","yards") and len(words) >= j+2:
            if words[j+1].rstrip(".,") in ("run","rush"):
                return int(words[j-1])
        
    return "x"
    
def yds_passed( detail ):
    words = detail.lower().split()
    # look for yardage in format "for X yards"
    for j, w in enumerate(words):
        if w == "for" and len(words) > j+2:
            if words[j+2].rstrip(".,") in ("yd","yds","yrd","yrds","yard","yards"):
                return int(words[j+1])
            # or "for no gain"
            elif "no" in words[j+1] and "gain" in words[j+2]:
                return 0
            
        # or "X yard pass"
        elif w in ("yd","yds","yrd","yrds","yard","yards") and len(words) >= j+2:
            if words[j+1].rstrip(".,") in ("pass"):
                return int(words[j-1])

    # Or maybe pass went incomplete
    if "incomplete" in detail.lower():
        return 0
    
    # Or maybe pass was intercepted. In this case, just say yds_gained is zero
    elif ("intercepted" in detail.lower()) or ("interception" in detail.lower()):
        return 0
    
    return "x"


def parse_details(df):
    # Make a bunch of lists to be populated with details in-loop    
    df_len = len(df.week.values)
    is_parseable = [False for i in range(df_len)]
    is_run = [False for i in range(df_len)]
    is_scramble = [False for i in range(df_len)]
    is_pass = [False for i in range(df_len)]
    is_punt = [False for i in range(df_len)]
    is_fieldgoal = [False for i in range(df_len)]
    is_kickoff = [False for i in range(df_len)]
    yds_gained = ['x' for i in range(df_len)]
    
    # Loop through details, use logic tree to classify plays
    for i, (down, 
            detail_text, 
            detail_plyrs) in enumerate(zip(df.down.values,
                                           df.detail_text.values,
                                           df.detail_href.values)):
        
        # Look for plays in downs 1-4
        if (str(down) != 'nan') and (str(down) != ''):
            
            detail_text = str(detail_text).lower()
#            print("parsing",detail_text)
            
            # Try and classify the play and parse yards gained
            if found_scramble(detail_text):
                is_scramble[i] = True
                yds_gained[i] = yds_run(detail_text)
                
            if found_run(detail_text):
                is_run[i] = True
                yds_gained[i] = yds_run(detail_text)
                
            if found_pass(detail_text):
                is_pass[i] = True
                yds_gained[i] = yds_passed(detail_text)
                
            elif found_punt(detail_text):
                is_punt[i] = True
                
            elif found_fieldgoal(detail_text):
                is_fieldgoal[i] = True
    
    for i, yds in enumerate(yds_gained):
        if (is_run[i] or is_pass[i] or is_scramble[i]) and (yds != 'x'):
            is_parseable[i] = True
        elif is_punt[i]:
            is_parseable[i] = True
        elif is_fieldgoal[i]:
            is_parseable[i] = True
            
    # Now write columns to the end of the DataFrame
    df['is_parseable'] = is_parseable
    df['is_run'] = is_run
    df['is_pass'] = is_pass
    df['is_scramble'] = is_scramble
    df['is_punt'] = is_punt
    df['is_fieldgoal'] = is_fieldgoal
    df['yds_gained'] = yds_gained
                            
    return df
    

def parse_pfr_pbp(df, gameinfo):
    """
    Function to extract additional information 
    from PFR-formatted play-by-play
    """
    (ssn, wk, gid, loc, home, away) = gameinfo    
    df['season'] = ssn
    df['week'] = wk
    df['gameid'] = gid
    df['home'] = home
    df['away'] = away
    df['loc'] = loc
    
    df['secs_rem'] = get_secs_rem(df)
    df['offense'] = get_possession(df,gameinfo)
    df['off_fieldpos'] = get_fieldposition(df,gameinfo)
    df = parse_details(df)
    
    return df

In [19]:
# Start looping over many games to get year, week, home, away info
season = 2017
filename = "../pfr_pages/team_season_pages_"+str(season)+".json"
with open(filename,'r') as f:
    season_pages = json.load(f)

# Start a list for individual game DataFrames
pbp_dfs = []

# Loop through individual games, parsing each DataFrame as we go
for tup in get_games(season_pages,'nwe2017')[:]:
    print(tup)
    (yr, wk, gid, loc, home, away) = tup
    if not gid == 'nan':
        game_df = pd.DataFrame(game_pages[gid]['pbp'])
        game_df = parse_pfr_pbp(game_df, tup)
        pbp_dfs.append(game_df)

cols = ['detail_a','detail_href','detail_text','offense',
        'location','off_fieldpos']
allplays_df = pd.concat(pbp_dfs)
allplays_df[cols].sample(20)

('2017', '1', '201709070nwe', '', 'nwe', 'kan')
('2017', '2', '201709170nor', '@', 'nor', 'nwe')
('2017', '3', '201709240nwe', '', 'nwe', 'htx')
('2017', '4', '201710010nwe', '', 'nwe', 'car')
('2017', '5', '201710050tam', '@', 'tam', 'nwe')
('2017', '6', '201710150nyj', '@', 'nyj', 'nwe')
('2017', '7', '201710220nwe', '', 'nwe', 'atl')
('2017', '8', '201710290nwe', '', 'nwe', 'sdg')
('2017', '9', 'nan', '', 'nwe', 'nan')
('2017', '10', '201711120den', '@', 'den', 'nwe')
('2017', '11', '201711190rai', '@', 'rai', 'nwe')
('2017', '12', '201711260nwe', '', 'nwe', 'mia')
('2017', '13', '201712030buf', '@', 'buf', 'nwe')
('2017', '14', '201712110mia', '@', 'mia', 'nwe')
('2017', '15', '201712170pit', '@', 'pit', 'nwe')
('2017', '16', '201712240nwe', '', 'nwe', 'buf')
('2017', '17', '201712310nwe', '', 'nwe', 'nyj')
('2017', '', 'nan', '', 'nwe', 'nan')
('2017', 'Division', '201801130nwe', '', 'nwe', 'oti')
('2017', 'Conf. Champ.', '201801210nwe', '', 'nwe', 'jax')
('2017', 'SuperBowl', '20

,detail_a,detail_href,detail_text,offense,location,off_fieldpos
30,"Tom Brady, James White, Mike Hilton","/players/B/BradTo00.htm, /players/W/WhitJa02.h...","pass complete short right to brk, for 4 yard...",nwe,NWE 27,-23
69,"Tom Brady, Rob Gronkowski, Micah Hyde","/players/B/BradTo00.htm, /players/G/GronRo00.h...","pass complete short right to brk, for 12 yar...",nwe,NWE 27,-23
152,"Derek Carr, Jared Cook","/players/C/CarrDe02.htm, /players/C/CookJa02.htm",pass incomplete short right intended for,rai,RAI 25,-25
121,"Jay Cutler, MarQueis Gray, Patrick Chung","/players/C/CutlJa00.htm, /players/G/GrayMa00.h...","pass complete short left to brk, for 10 yard...",mia,MIA 15,-35
113,"Dion Lewis, Preston Brown, Kyle Williams","/players/L/LewiDi00.htm, /players/B/BrowPr00.h...","middle for 3 yards (tackle by brk, and brk, )",nwe,BUF 29,21
149,"Tom Brady, Danny Amendola, Logan Ryan","/players/B/BradTo00.htm, /players/A/AmenDa00.h...","pass complete short left to brk, for 12 yard...",nwe,OTI 28,22
103,Stephen Gostkowski,/players/G/GostSt20.htm,kicks extra point good,nwe,MIA 15,35
58,"Le'Veon Bell, Malcom Brown, Eric Lee","/players/B/BellLe00.htm, /players/B/BrowMa01.h...","left tackle for 1 yard (tackle by brk, and b...",pit,PIT 32,-18
40,"Philip Rivers, Keenan Allen, Keenan Allen","/players/R/RivePh00.htm, /players/A/AlleKe00.h...","pass incomplete deep left intended for brk, ....",sdg,LAC 15,35
6,Melvin Gordon,/players/G/GordMe00.htm,left tackle for 2 yards,sdg,NWE 46,4


In [20]:
cols = ['detail_text','is_parseable','is_pass','is_run','is_scramble',
        'is_punt','is_fieldgoal','yds_gained']
allplays_df[cols].sample(20)

,detail_text,is_parseable,is_pass,is_run,is_scramble,is_punt,is_fieldgoal,yds_gained
33,pass incomplete,True,True,False,False,False,False,0
16,punts 39 yards,True,False,False,False,True,False,x
146,"pass complete short right to brk, for -8 yar...",True,True,False,False,False,False,-8
155,"pass complete short left to brk, for 18 yard...",True,True,False,False,False,False,18
85,58 yard field goal good,True,False,False,False,False,True,x
36,"pass complete short left to brk, for 1 yard ...",True,True,False,False,False,False,1
115,"middle for no gain (tackle by brk, )",False,False,False,False,False,False,x
11,"pass complete short right to brk, for 16 yar...",True,True,False,False,False,False,16
170,"left tackle for 2 yards (tackle by brk, )",True,False,True,False,False,False,2
158,"middle for 8 yards, touchdown",False,False,False,False,False,False,x


## Looks good!

In [21]:
# Parse all plays for each season
pbp_dfs = []
for ssn in range(2003,2018):
    # Start looping over many games to get year, week, home, away info
    filename = "../pfr_pages/team_season_pages_"+str(ssn)+".json"
    with open(filename,'r') as f:
        season_pages = json.load(f)

    # Loop through individual games, parsing each DataFrame as we go
    parsed_games = []
    for teamcode in season_pages.keys():
        for tup in get_games(season_pages,teamcode)[:]:
            (yr, wk, gid, loc, home, away) = tup
            if (gid != 'nan') and (gid not in parsed_games):
                try:
                    print(tup)
                    game_df = pd.DataFrame(game_pages[gid]['pbp'])
                    game_df = parse_pfr_pbp(game_df, tup)
                    pbp_dfs.append(game_df)
                    parsed_games.append(gid)
                except:
                    print("Parsing",gid,"went wonky")

cols = ['detail_a','detail_href','detail_text','offense',
        'location','off_fieldpos']
allplays_df = pd.concat(pbp_dfs)
allplays_df[cols].sample(20)

('2003', '1', '200309070buf', '@', 'buf', 'nwe')
Couldn't match player /players/v/vinatada01.htm
('2003', '2', '200309140phi', '@', 'phi', 'nwe')
('2003', '3', '200309210nwe', '', 'nwe', 'nyj')
('2003', '4', '200309280was', '@', 'was', 'nwe')
('2003', '5', '200310050nwe', '', 'nwe', 'oti')
('2003', '6', '200310120nwe', '', 'nwe', 'nyg')
('2003', '7', '200310190mia', '@', 'mia', 'nwe')
('2003', '8', '200310260nwe', '', 'nwe', 'cle')
('2003', '9', '200311030den', '@', 'den', 'nwe')
Couldn't match player /players/w/woodda20.htm
('2003', '11', '200311160nwe', '', 'nwe', 'dal')
('2003', '12', '200311230htx', '@', 'htx', 'nwe')
Couldn't match player /players/m/mayesh20.htm
('2003', '13', '200311300clt', '@', 'clt', 'nwe')
('2003', '14', '200312070nwe', '', 'nwe', 'mia')
Couldn't match player /players/m/marexoli01.htm
('2003', '15', '200312140nwe', '', 'nwe', 'jax')
Couldn't match player /players/s/spicpa20.htm
('2003', '16', '200312200nyj', '@', 'nyj', 'nwe')
Couldn't match player /players/v

Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
('2003', '14', '200312070rav', '', 'rav', 'cin')
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/rehbsc20.htm
Couldn't match player /players/r/richewad01.htm
('2003', '15', '200312140rai', '@', 'rai', 'rav')
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/richewad01.htm
('2003', '16', '200312210cle', '@', 'cle', 'rav')
Couldn't match player /players/r/richewad01.htm
Couldn't match player /players/r/rich

Couldn't match player /teams/kan/2003.htm
('2003', '6', '200310120gnb', '@', 'gnb', 'kan')
('2003', '7', '200310200rai', '@', 'rai', 'kan')
('2003', '12', '200311230kan', '', 'kan', 'rai')
('2003', '13', '200311300sdg', '@', 'sdg', 'kan')
Couldn't match player /players/s/scifmi20.htm
Couldn't match player /players/a/andermor01.htm
Couldn't match player /players/a/andermor01.htm
Couldn't match player /players/s/scifmi20.htm
Couldn't match player /players/a/andermor01.htm
Couldn't match player /players/s/scifmi20.htm
Couldn't match player /players/s/scifmi20.htm
Couldn't match player /players/a/andermor01.htm
('2003', '14', '200312070den', '@', 'den', 'kan')
Couldn't match player /players/a/andermor01.htm
Couldn't match player /players/a/andermor01.htm
Couldn't match player /players/a/andermor01.htm
('2003', '15', '200312140kan', '', 'kan', 'det')
('2003', '16', '200312200min', '@', 'min', 'kan')
Couldn't match player /players/a/andermor01.htm
Couldn't match player /players/a/andermor01.

Couldn't match player /players/g/gramamar01.htm
Couldn't match player /players/g/gramamar01.htm
Couldn't match player /players/g/gramamar01.htm
('2003', '10', '200311090was', '', 'was', 'sea')
('2003', '11', '200311160car', '@', 'car', 'was')
('2003', '13', '200311300was', '', 'was', 'nor')
('2003', '14', '200312070nyg', '@', 'nyg', 'was')
('2003', '16', '200312210chi', '@', 'chi', 'was')
('2003', '1', '200309070nyg', '', 'nyg', 'ram')
('2003', '8', '200310260min', '@', 'min', 'nyg')
Couldn't match player /players/c/cortejos01.htm
Couldn't match player /players/c/cortejos01.htm
Couldn't match player /players/c/cortejos01.htm
Couldn't match player /players/c/cortejos01.htm
('2003', '10', '200311090nyg', '', 'nyg', 'atl')
Couldn't match player /players/c/conwabre01.htm
Couldn't match player /players/f/feelyjay01.htm
Couldn't match player /players/f/feelyjay01.htm
Couldn't match player /players/c/conwabre01.htm
Couldn't match player /players/c/conwabre01.htm
Couldn't match player /players

('2003', '9', '200311020sfo', '@', 'sfo', 'ram')
Couldn't match player /players/m/manubr00.htm
('2003', '12', '200311230crd', '@', 'crd', 'ram')
Couldn't match player /players/d/davile21.htm
('2003', '15', '200312140ram', '', 'ram', 'sea')
('2003', '2', '200309140crd', '@', 'crd', 'sea')
('2003', '6', '200310120sea', '', 'sea', 'sfo')
('2003', '16', '200312210sea', '', 'sea', 'crd')
Couldn't match player /players/b/brownjos01.htm
Couldn't match player /players/b/brownjos01.htm
Couldn't match player /players/b/brownjos01.htm
Couldn't match player /players/b/brownjos01.htm
Couldn't match player /players/b/brownjos01.htm
Couldn't match player /players/b/brownjos01.htm
Couldn't match player /players/s/spikca20.htm
Couldn't match player /players/b/brownjos01.htm
Couldn't match player /players/b/brownjos01.htm
Couldn't match player /players/b/brownjos01.htm
('2003', '17', '200312270sfo', '@', 'sfo', 'sea')
('2003', '8', '200310260crd', '@', 'crd', 'sfo')
('2003', '14', '200312070sfo', '', 's

...

('2015', '5', '201510110det', '', 'det', 'crd')
('2015', '6', '201510180det', '', 'det', 'chi')
('2015', '14', '201512130ram', '@', 'ram', 'det')
('2015', '15', '201512210nor', '@', 'nor', 'det')
('2015', '16', '201512270det', '', 'det', 'sfo')
('2015', '17', '201601030chi', '@', 'chi', 'det')
('2015', '2', '201509200chi', '', 'chi', 'crd')
('2015', '3', '201509270sea', '@', 'sea', 'chi')
('2015', '10', '201511150ram', '@', 'ram', 'chi')
('2015', '13', '201512060chi', '', 'chi', 'sfo')
('2015', '16', '201512270tam', '@', 'tam', 'chi')
('2015', '3', '201509270car', '', 'car', 'nor')
('2015', '4', '201510040tam', '@', 'tam', 'car')
('2015', '6', '201510180sea', '@', 'sea', 'car')
('2015', '13', '201512060nor', '@', 'nor', 'car')
('2015', '14', '201512130car', '', 'car', 'atl')
('2015', '16', '201512270atl', '@', 'atl', 'car')
('2015', '17', '201601030car', '', 'car', 'tam')
('2015', 'Division', '201601170car', '', 'car', 'sea')
('2015', 'Conf. Champ.', '201601240car', '', 'car', 'crd')
(

('2016', '3', '201609250clt', '', 'clt', 'sdg')
('2016', '4', '201610020jax', '@', 'jax', 'clt')
('2016', '5', '201610090clt', '', 'clt', 'chi')
('2016', '8', '201610300clt', '', 'clt', 'kan')
('2016', '9', '201611060gnb', '@', 'gnb', 'clt')
('2016', '15', '201612180min', '@', 'min', 'clt')
('2016', '16', '201612240rai', '@', 'rai', 'clt')
('2016', '17', '201701010clt', '', 'clt', 'jax')
('2016', '1', '201609110jax', '', 'jax', 'gnb')
('2016', '2', '201609180sdg', '@', 'sdg', 'jax')
('2016', '6', '201610160chi', '@', 'chi', 'jax')
('2016', '7', '201610230jax', '', 'jax', 'rai')
('2016', '9', '201611060kan', '@', 'kan', 'jax')
('2016', '11', '201611200det', '@', 'det', 'jax')
('2016', '13', '201612040jax', '', 'jax', 'den')
('2016', '14', '201612110jax', '', 'jax', 'min')
('2016', '1', '201609110kan', '', 'kan', 'sdg')
('2016', '6', '201610160rai', '@', 'rai', 'kan')
('2016', '7', '201610230kan', '', 'kan', 'nor')
('2016', '10', '201611130car', '@', 'car', 'kan')
('2016', '11', '2016112

('2017', '2', '201709170sdg', '@', 'sdg', 'mia')
('2017', '3', '201709240nyj', '@', 'nyj', 'mia')
('2017', '4', '201710010mia', '', 'mia', 'nor')
('2017', '5', '201710080mia', '', 'mia', 'oti')
('2017', '6', '201710150atl', '@', 'atl', 'mia')
('2017', '7', '201710220mia', '', 'mia', 'nyj')
('2017', '8', '201710260rav', '@', 'rav', 'mia')
('2017', '9', '201711050mia', '', 'mia', 'rai')
('2017', '10', '201711130car', '@', 'car', 'mia')
('2017', '11', '201711190mia', '', 'mia', 'tam')
('2017', '13', '201712030mia', '', 'mia', 'den')
Couldn't match player /teams/den/2017.htm
('2017', '16', '201712240kan', '@', 'kan', 'mia')
('2017', '2', '201709170rai', '@', 'rai', 'nyj')
('2017', '4', '201710010nyj', '', 'nyj', 'jax')
('2017', '5', '201710080cle', '@', 'cle', 'nyj')
('2017', '8', '201710290nyj', '', 'nyj', 'atl')
('2017', '10', '201711120tam', '@', 'tam', 'nyj')
('2017', '12', '201711260nyj', '', 'nyj', 'car')
('2017', '13', '201712030nyj', '', 'nyj', 'kan')
('2017', '14', '201712100den',

('2017', '9', '201711050nyg', '', 'nyg', 'ram')
('2017', '10', '201711120sfo', '@', 'sfo', 'nyg')
('2017', '16', '201712240crd', '@', 'crd', 'nyg')
('2017', '1', '201709110min', '', 'min', 'nor')
('2017', '3', '201709240min', '', 'min', 'tam')
('2017', '4', '201710010min', '', 'min', 'det')
('2017', '5', '201710090chi', '@', 'chi', 'min')
('2017', '6', '201710150min', '', 'min', 'gnb')
('2017', '11', '201711190min', '', 'min', 'ram')
('2017', '12', '201711230det', '@', 'det', 'min')
('2017', '13', '201712030atl', '@', 'atl', 'min')
('2017', '14', '201712100car', '@', 'car', 'min')
('2017', '16', '201712230gnb', '@', 'gnb', 'min')
('2017', '17', '201712310min', '', 'min', 'chi')
('2017', 'Division', '201801140min', '', 'min', 'nor')
('2017', '1', '201709100det', '', 'det', 'crd')
('2017', '3', '201709240det', '', 'det', 'atl')
('2017', '5', '201710080det', '', 'det', 'car')
Couldn't match player /teams/car/2017.htm
('2017', '6', '201710150nor', '@', 'nor', 'det')
('2017', '9', '20171106

,detail_a,detail_href,detail_text,offense,location,off_fieldpos
5,"Kyle Orton, Eddie Royal, Anthony Smith","/players/O/OrtoKy00.htm, /players/R/RoyaEd00.h...","pass complete short middle to brk, for 13 ya...",jax,JAX 40,-10
136,"Sam Koch, Dez Bryant, Anthony Allen","/players/K/KochSa20.htm, /players/B/BryaDe01.h...","punts 37 yards, returned by brk, for 10 yard...",rav,RAV 10,-40
53,"Alfred Morris, Mark Barron, Gerald McCoy","/players/M/MorrAl00.htm, /players/B/BarrMa00.h...","up the middle for 7 yards (tackle by brk, ). ...",was,TAM 26,24
68,Joe Nedney,/players/N/nednejoe01.htm,28 yard field goal good,sfo,SEA 10,40
91,Brian Hoyer,/players/H/HoyeBr00.htm,kneels for -1 yards,chi,CHI 25,-25
121,"Jake Plummer, Raheem Brock","/players/P/PlumJa00.htm, /players/B/BrocRa20.htm","sacked by brk, for -7 yards",den,CLT 26,24
185,"Eli Manning, Odell Beckham, Darius Butler","/players/M/MannEl00.htm, /players/B/BeckOd00.h...","pass complete deep left to brk, for 18 yards...",nyg,CLT 47,3
102,"Tom Brady, Danny Amendola, Trent Cole","/players/B/BradTo00.htm, /players/A/AmenDa00.h...","pass complete short left to brk, for 11 yard...",nwe,NWE 20,-30
72,"Doug Martin, Jared Allen","/players/M/MartDo00.htm, /players/A/AlleJa22.htm","right guard for 2 yards (tackle by brk, )",tam,TAM 20,-30
22,"Frank Gore, Karlos Dansby","/players/G/GoreFr00.htm, /players/D/DansKa20.htm","right guard for 36 yards (tackle by brk, )",sfo,SFO 27,-23


In [22]:
info = ('2013', '4', '201309300nor', '@', 'nor', 'mia')
game_df = pd.DataFrame(game_pages['201309300nor']['pbp'])
parse_pfr_pbp(game_df, info)

,detail,detail_a,detail_href,detail_text,down,exp_pts_after,exp_pts_before,home_wp,location,onecell,...,secs_rem,offense,off_fieldpos,is_parseable,is_run,is_pass,is_scramble,is_punt,is_fieldgoal,yds_gained
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1st Quarter,...,3600,nor,0,False,False,False,False,False,False,x
1,NaN,"Caleb Sturgis, Darren Sproles, Jimmy Wilson","/players/S/SturCa00.htm, /players/S/SproDa00.h...","kicks off 71 yards, returned by brk, for 24 ...",,-0.040,0.000,71.2,MIA 35,NaN,...,3600,mia,-15,False,False,False,False,False,False,x
2,NaN,"Pierre Thomas, Nolan Carroll, Philip Wheeler","/players/T/ThomPi00.htm, /players/C/CarrNo00.h...","left end for 3 yards (tackle by brk, and brk, )",1,0.010,0.040,71.1,NOR 18,NaN,...,3596,nor,-32,True,True,False,False,False,False,3
3,NaN,"Drew Brees, Darren Sproles, Chris Clemons","/players/B/BreeDr00.htm, /players/S/SproDa00.h...","pass complete deep left to brk, for 48 yards...",2,3.510,0.010,79.3,NOR 21,NaN,...,3561,nor,-29,True,False,True,False,False,False,48
4,NaN,"Drew Brees, Darren Sproles, Dannell Ellerbe","/players/B/BreeDr00.htm, /players/S/SproDa00.h...","pass complete short right to brk, for 4 yard...",1,3.510,3.510,79.3,MIA 31,NaN,...,3530,nor,19,True,False,True,False,False,False,4
5,NaN,"Khiry Robinson, Derrick Shelby","/players/R/RobiKh00.htm, /players/S/ShelDe00.htm","right tackle for 4 yards (tackle by brk, )",2,3.340,3.510,78.8,MIA 27,NaN,...,3495,nor,23,False,False,False,False,False,False,x
6,NaN,"Drew Brees, Kenny Stills, Chris Clemons","/players/B/BreeDr00.htm, /players/S/StilKe00.h...","pass complete short middle to brk, for 18 ya...",3,6.060,3.340,84.3,MIA 23,NaN,...,3451,nor,27,True,False,True,False,False,False,18
7,NaN,Darren Sproles,/players/S/SproDa00.htm,"left guard for 5 yards, touchdown",1,7.000,6.060,84.1,MIA 5,NaN,...,3416,nor,45,True,True,False,False,False,False,5
8,NaN,Garrett Hartley,/players/H/HartGa00.htm,kicks extra point good,,0.000,0.000,85.9,,NaN,...,3416,nor,0,False,False,False,False,False,False,x
9,NaN,"Thomas Morstead, Marcus Thigpen, Khiry Robinson","/players/M/MorsTh00.htm, /players/T/ThigMa00.h...","kicks off 64 yards, returned by brk, for 16 ...",,0.060,0.000,86.0,NOR 35,NaN,...,3416,nor,-15,False,False,False,False,False,False,x


## Save play-by-play DataFrame as its own file

In [23]:
allplays_df.to_csv("../data/pfr_pbp2003-2017.csv")

MemoryError: 

# Figure out whether a play was a "success"
1st down - 45%

2nd down - 65%

3rd/4th down - 100%

# Make a DataFrame of individual players' game stats

In [ ]:
gid = '201309300nor'
pd.DataFrame(game_pages[gid]['player_offense']).columns

In [ ]:
# Parse all plays for each season
ind_statlines = []
for ssn in range(2003,2018):
    # Start looping over many games to get year, week, home, away info
    filename = "../pfr_pages/team_season_pages_"+str(ssn)+".json"
    with open(filename,'r') as f:
        season_pages = json.load(f)

    # Loop through individual games, parsing each DataFrame as we go
    parsed_games = []
    for teamcode in season_pages.keys():
        for tup in get_games(season_pages,teamcode)[:]:
            (yr, wk, gid, loc, home, away) = tup
            if (gid != 'nan') and (gid not in parsed_games):
                try:
                    game_df = pd.DataFrame(game_pages[gid]['player_offense'])
                    game_df['gid'] = gid
                    game_df['season'] = yr
                    game_df['week'] = wk
                    game_df['home'] = home
                    game_df['away'] = away
                    ind_statlines.append(game_df)
                    parsed_games.append(gid)
                except:
                    print("Parsing",gid,"went wonky")

cols = ['player_href', 'player_a', 'team', 'gid']
ind_stats_df = pd.concat(ind_statlines)
ind_stats_df[cols].sample(20)

In [ ]:
ind_stats_df.to_csv("../data/pfr_ind_off_stats2003-2017.csv")

In [3]:
ind_stats_df = pd.read_csv("../data/pfr_ind_off_stats2003-2017.csv")

In [4]:
ind_stats_df[
    ind_stats_df['player_a'] == 'Marqise Lee'
]

,Unnamed: 0,Unnamed: 1,fumbles,fumbles_lost,header_fumbles,header_pass,header_rec,header_rush,pass_att,pass_cmp,...,rush_long,rush_td,rush_yds,targets,team,gid,season,week,home,away
62053,20,NaN,0,0,NaN,NaN,NaN,NaN,0,0,...,0,0,0,2,JAX,201410260jax,2014,8,jax,mia
63275,9,NaN,0,0,NaN,NaN,NaN,NaN,0,0,...,0,0,0,10,JAX,201412140rav,2014,15,rav,jax
63397,16,NaN,0,0,NaN,NaN,NaN,NaN,0,0,...,5,0,5,3,JAX,201410190jax,2014,7,jax,cle
63666,3,NaN,0,0,NaN,NaN,NaN,NaN,0,0,...,0,0,0,5,JAX,201411230clt,2014,12,clt,jax
63959,14,NaN,0,0,NaN,NaN,NaN,NaN,0,0,...,4,0,4,8,JAX,201412070jax,2014,14,jax,htx
63971,6,NaN,0,0,NaN,NaN,NaN,NaN,0,0,...,0,0,0,8,JAX,201412280htx,2014,17,htx,jax
63989,5,NaN,0,0,NaN,NaN,NaN,NaN,0,0,...,0,0,0,10,JAX,201409070phi,2014,1,phi,jax
64011,6,NaN,0,0,NaN,NaN,NaN,NaN,0,0,...,0,0,0,4,JAX,201409140was,2014,2,was,jax
64055,7,NaN,0,0,NaN,NaN,NaN,NaN,0,0,...,0,0,0,2,JAX,201410120oti,2014,6,oti,jax
64092,21,NaN,0,0,NaN,NaN,NaN,NaN,0,0,...,0,0,0,3,JAX,201411090jax,2014,10,jax,dal


In [5]:
rec_cols = ['season','team','week','player_a','targets','rec','rec_yds',
             'rec_td','rec_long','fumbles']
rb_cols = ['season','team','week','player_a','rush_att','rush_yds',
           'rush_td','rush_long','targets','rec','rec_yds','rec_td',
           'fumbles']
qb_cols = ['season','team','week','player_a','pass_att','pass_cmp',
           'pass_yds','pass_td','pass_int','pass_long','pass_rating',
           'pass_sacked','pass_sacked_yds','rush_att','rush_yds','rush_td',
           'rush_long']

In [6]:
ind_stats_df[
    ind_stats_df['player_a'] == 'Devin Funchess'
][rec_cols].sort_values(by=['season','week'],ascending=True)

,season,team,week,player_a,targets,rec,rec_yds,rec_td,rec_long,fumbles
69793,2015,CAR,1,Devin Funchess,2,1,9,0,9,0
70007,2015,CAR,10,Devin Funchess,2,2,41,0,27,0
70834,2015,CAR,11,Devin Funchess,8,4,64,1,35,0
71482,2015,CAR,12,Devin Funchess,4,2,19,0,15,0
72335,2015,CAR,13,Devin Funchess,3,1,13,1,13,0
72371,2015,CAR,14,Devin Funchess,5,3,25,0,11,0
71348,2015,CAR,15,Devin Funchess,7,2,30,1,16,0
72383,2015,CAR,16,Devin Funchess,1,0,0,0,0,0
72409,2015,CAR,17,Devin Funchess,8,7,120,1,41,1
69357,2015,CAR,2,Devin Funchess,4,1,15,0,15,0


In [7]:
ind_stats_df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'fumbles', 'fumbles_lost', 'header_fumbles',
       'header_pass', 'header_rec', 'header_rush', 'pass_att', 'pass_cmp',
       'pass_int', 'pass_long', 'pass_rating', 'pass_sacked',
       'pass_sacked_yds', 'pass_td', 'pass_yds', 'player', 'player_a',
       'player_href', 'rec', 'rec_long', 'rec_td', 'rec_yds', 'rowclass',
       'rush_att', 'rush_long', 'rush_td', 'rush_yds', 'targets', 'team',
       'gid', 'season', 'week', 'home', 'away'],
      dtype='object')

In [8]:
ind_stats_df = ind_stats_df.drop(columns=['header_fumbles','header_pass',
                                          'header_rec','header_rush',
                                          'rowclass'])

In [9]:
ind_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84559 entries, 0 to 84558
Data columns (total 31 columns):
Unnamed: 0         84559 non-null int64
Unnamed: 1         0 non-null float64
fumbles            80546 non-null object
fumbles_lost       35747 non-null object
pass_att           80554 non-null object
pass_cmp           80554 non-null object
pass_int           80546 non-null object
pass_long          80546 non-null object
pass_rating        13668 non-null object
pass_sacked        48965 non-null object
pass_sacked_yds    47521 non-null object
pass_td            80554 non-null object
pass_yds           80554 non-null object
player             4005 non-null object
player_a           76549 non-null object
player_href        76549 non-null object
rec                80554 non-null object
rec_long           80552 non-null object
rec_td             80554 non-null object
rec_yds            80554 non-null object
rush_att           80554 non-null object
rush_long          80546 non-null o

In [10]:
# Drop unusable rows.
ind_stats_df = ind_stats_df[ ind_stats_df.player.isnull() ]

In [11]:
# Show that there's nothing in this subset of rows
ind_stats_df[ ind_stats_df.player_a.isnull() ].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4005 entries, 11 to 84547
Data columns (total 31 columns):
Unnamed: 0         4005 non-null int64
Unnamed: 1         0 non-null float64
fumbles            0 non-null object
fumbles_lost       0 non-null object
pass_att           0 non-null object
pass_cmp           0 non-null object
pass_int           0 non-null object
pass_long          0 non-null object
pass_rating        0 non-null object
pass_sacked        0 non-null object
pass_sacked_yds    0 non-null object
pass_td            0 non-null object
pass_yds           0 non-null object
player             0 non-null object
player_a           0 non-null object
player_href        0 non-null object
rec                0 non-null object
rec_long           0 non-null object
rec_td             0 non-null object
rec_yds            0 non-null object
rush_att           0 non-null object
rush_long          0 non-null object
rush_td            0 non-null object
rush_yds           0 non-null object


In [12]:
ind_stats_df = ind_stats_df[ ~ind_stats_df.player_a.isnull() ]

In [13]:
ind_stats_df = ind_stats_df.drop(columns=['player'])

In [14]:
ind_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76549 entries, 0 to 84558
Data columns (total 30 columns):
Unnamed: 0         76549 non-null int64
Unnamed: 1         0 non-null float64
fumbles            76541 non-null object
fumbles_lost       31742 non-null object
pass_att           76549 non-null object
pass_cmp           76549 non-null object
pass_int           76541 non-null object
pass_long          76541 non-null object
pass_rating        9663 non-null object
pass_sacked        44960 non-null object
pass_sacked_yds    43516 non-null object
pass_td            76549 non-null object
pass_yds           76549 non-null object
player_a           76549 non-null object
player_href        76549 non-null object
rec                76549 non-null object
rec_long           76547 non-null object
rec_td             76549 non-null object
rec_yds            76549 non-null object
rush_att           76549 non-null object
rush_long          76541 non-null object
rush_td            76549 non-null o

In [15]:
reg_season = [str(x+1) for x in range(17)]
b_list = [ True if x in reg_season else False for x in ind_stats_df.week ]
ind_stats_df = ind_stats_df[ b_list ]

In [16]:
[ 0 if x=='' else x for x in ind_stats_df.pass_yds ]

['123',
 '31',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '230',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '255',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '186',
 '25',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '264',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '181',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '289',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '147',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '360',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '219',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '314',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '112',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '283',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '230',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '115',
 '40',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '259',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '350',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0

In [29]:
[str(x) for x in ind_stats_df['pass_int'].unique()]

['4', '0', '1', '2', '3', '5', 'nan', '6']

In [35]:
int_cols = [
    'week', 'season', 'pass_att', 'pass_cmp', 'pass_int', 'pass_long',
    'pass_sacked', 'pass_sacked_yds','pass_td','pass_yds','rec','rec_long',
    'rec_td','rec_yds','rush_att','rush_long','rush_td','rush_yds',
    'targets','fumbles','fumbles_lost'
]
for c in int_cols:
    ind_stats_df[c] = [0 if (x=='' or str(x)=='nan') else x for x in ind_stats_df[c]]
    ind_stats_df[c] = ind_stats_df[c].astype(int)

ind_stats_df.sort_values(by=['season','week','team'],
                         ascending=False,
                         inplace=True)

ind_stats_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 73454 entries, 83039 to 4643
Data columns (total 30 columns):
Unnamed: 0         73454 non-null int64
Unnamed: 1         0 non-null float64
fumbles            73454 non-null int64
fumbles_lost       73454 non-null int64
pass_att           73454 non-null int64
pass_cmp           73454 non-null int64
pass_int           73454 non-null int64
pass_long          73454 non-null int64
pass_rating        73454 non-null object
pass_sacked        73454 non-null int64
pass_sacked_yds    73454 non-null int64
pass_td            73454 non-null int64
pass_yds           73454 non-null int64
player_a           73454 non-null object
player_href        73454 non-null object
rec                73454 non-null int64
rec_long           73454 non-null int64
rec_td             73454 non-null int64
rec_yds            73454 non-null int64
rush_att           73454 non-null int64
rush_long          73454 non-null int64
rush_td            73454 non-null int64
rush_yd

In [33]:
ind_stats_df[rb_cols].head(35)

,season,team,week,player_a,rush_att,rush_yds,rush_td,rush_long,targets,rec,rec_yds,rec_td,fumbles
83039,2017,WAS,17,Kirk Cousins,1,12,1,12,0,0,0,0,0
83040,2017,WAS,17,Kapri Bibbs,12,47,0,16,9,8,53,0,0
83041,2017,WAS,17,Samaje Perine,2,3,0,4,0,0,0,0,0
83042,2017,WAS,17,Josh Doctson,0,0,0,0,10,4,37,0,0
83043,2017,WAS,17,Ryan Grant,0,0,0,0,5,2,23,0,0
83044,2017,WAS,17,Vernon Davis,0,0,0,0,3,2,20,0,0
83045,2017,WAS,17,Brian Quick,0,0,0,0,1,0,0,0,0
83046,2017,WAS,17,Niles Paul,0,0,0,0,2,1,0,0,0
83047,2017,WAS,17,Jamison Crowder,1,-1,0,-1,7,3,25,0,0
81245,2017,TEN,17,Marcus Mariota,10,60,0,15,0,0,0,0,1


In [36]:
ind_stats_df.to_csv("../data/ind_off_regseason.csv")

In [43]:
def single_game(df, pid, season, week):
    result = df[
        (df.player_href == pid) &
        (df.season == season) &
        (df.week == week)
    ]
    return result

def multi_game(df, pid, yr_start, yr_end, wk_start, wk_end):
    result = df[
        (df.player_href == pid) &
        (df.season >= yr_start) &
        (df.season <= yr_end) &
        (df.week >= wk_start) &
        (df.week <= wk_end)
    ]
    return result

def yr_to_date(df, pid, yr, current_wk):
    result = df[
        (df.player_href == pid) &
        (df.season == yr) &
        (df.week < current_wk)
    ]
    return result

def prev_season(df, pid, current_yr):
    result = df[
        (df.player_href == pid) &
        (df.season == current_yr-1)
    ]
    return result

# For a stat, get previous performance in that stat
def prev_perf(df, row):
    pid = row['player_href'].values[0]
    ssn = row['season'].values[0]
    wk = row['week'].values[0]
    
    # get df of previous games
    prev = df[ 
        (df.player_href==pid) &
        (
            (df.season < ssn) |
            (df.week < wk)
        )
    ]
    prev = prev.sort_values( by=['season','week'], ascending=[False,False] )
    
    return prev


row = ind_stats_df[
    (ind_stats_df.player_a == 'Jonathan Stewart') &
    (ind_stats_df.season == 2017) &
    (ind_stats_df.week == 16)
]

prev_perf(ind_stats_df, row)[rb_cols].head(5).agg('mean')

season       2017.0
week           12.8
rush_att       14.0
rush_yds       62.2
rush_td         1.0
rush_long      20.2
targets         0.6
rec             0.2
rec_yds         1.8
rec_td          0.0
fumbles         0.0
dtype: float64

## Make a column for snaps

In [44]:
# Get snap counts for particular game
gid = '201612180min'
snaps_tables = [
    pd.DataFrame(game_pages[gid][x]) for x in ('home_snap_counts',
                                               'vis_snap_counts')
]
snaps_df = pd.concat(snaps_tables)
off_snaps = snaps_df.set_index('player_href')

game_df = ind_stats_df[ ind_stats_df.gid == gid ]

ind_game_w_snaps = pd.merge(game_df, 
                            off_snaps, 
                            how='left',
                            left_on='player_href', 
                            right_index=True)
ind_game_w_snaps.drop(columns=['player_a_y', 'def_pct','defense'])

,fumbles,fumbles_lost,pass_att,pass_cmp,pass_int,pass_long,pass_rating,pass_sacked,pass_sacked_yds,pass_td,...,gid,season,week,home,away,off_pct,offense,pos,special_teams,st_pct
13,1,1,42,32,1,28,84.5,5,43,0,...,201612180min,2016,15,min,clt,100%,58,QB,0,0%
14,1,1,0,0,0,0,,0,0,0,...,201612180min,2016,15,min,clt,21%,12,RB,0,0%
15,0,0,0,0,0,0,,0,0,0,...,201612180min,2016,15,min,clt,59%,34,RB,0,0%
16,0,0,0,0,0,0,,0,0,0,...,201612180min,2016,15,min,clt,97%,56,TE,2,8%
17,0,0,0,0,0,0,,0,0,0,...,201612180min,2016,15,min,clt,67%,39,WR,13,54%
18,0,0,0,0,0,0,,0,0,0,...,201612180min,2016,15,min,clt,76%,44,WR,9,38%
19,0,0,0,0,0,0,,0,0,0,...,201612180min,2016,15,min,clt,21%,12,RB,4,17%
20,0,0,0,0,0,0,,0,0,0,...,201612180min,2016,15,min,clt,84%,49,WR,0,0%
21,0,0,0,0,0,0,,0,0,0,...,201612180min,2016,15,min,clt,29%,17,WR,0,0%
22,0,0,0,0,0,0,,0,0,0,...,201612180min,2016,15,min,clt,31%,18,WR,5,21%


In [45]:
def get_snaps(gid):
    snaps_tables = [
        pd.DataFrame(game_pages[gid][x]) for x in ('home_snap_counts',
                                                   'vis_snap_counts')
    ]
    snaps_df = pd.concat(snaps_tables)
    off_snaps = snaps_df.set_index('player_href')
    
    game_df = ind_stats_df[ ind_stats_df.gid == gid ]
    
    ind_game_w_snaps = pd.merge(game_df, 
                                off_snaps, 
                                how='left',
                                left_on='player_href', 
                                right_index=True)
    ind_game_w_snaps.drop(columns=['player_a_y', 'def_pct','defense'],
                          inplace=True)
    
    return ind_game_w_snaps

In [46]:
ind_game_dfs = []
for gid in ind_stats_df.gid.unique():
    if 'home_snap_counts' in game_pages[gid].keys():
        print("Found snaps for",gid)
        game_df = get_snaps(gid)
        ind_game_dfs.append(game_df)

ind_stats_w_snaps_df = pd.concat(ind_game_dfs, sort=True)
ind_stats_w_snaps_df.sample(5)

Found snaps for 201712310nyg
Found snaps for 201712310oti
Found snaps for 201712310tam
Found snaps for 201712310ram
Found snaps for 201712310sea
Found snaps for 201712310pit
Found snaps for 201712310phi
Found snaps for 201712310sdg
Found snaps for 201712310nwe
Found snaps for 201712310min
Found snaps for 201712310mia
Found snaps for 201712310den
Found snaps for 201712310clt
Found snaps for 201712310det
Found snaps for 201712310rav
Found snaps for 201712310atl
Found snaps for 201712240was
Found snaps for 201712240oti
Found snaps for 201712240car
Found snaps for 201712240sfo
Found snaps for 201712240dal
Found snaps for 201712250htx
Found snaps for 201712250phi
Found snaps for 201712240nyj
Found snaps for 201712240crd
Found snaps for 201712240nwe
Found snaps for 201712240nor
Found snaps for 201712230gnb
Found snaps for 201712240kan
Found snaps for 201712230rav
Found snaps for 201712240cin
Found snaps for 201712240chi
Found snaps for 201712170was
Found snaps for 201712170sfo
Found snaps fo

Found snaps for 201612180min
Found snaps for 201612180htx
Found snaps for 201612180chi
Found snaps for 201612180buf
Found snaps for 201612110phi
Found snaps for 201612110oti
Found snaps for 201612110tam
Found snaps for 201612110sfo
Found snaps for 201612110gnb
Found snaps for 201612110car
Found snaps for 201612110buf
Found snaps for 201612080kan
Found snaps for 201612110nyg
Found snaps for 201612120nwe
Found snaps for 201612110jax
Found snaps for 201612110mia
Found snaps for 201612110ram
Found snaps for 201612110clt
Found snaps for 201612110det
Found snaps for 201612110cle
Found snaps for 201612040crd
Found snaps for 201612040sdg
Found snaps for 201612040chi
Found snaps for 201612040sea
Found snaps for 201612040pit
Found snaps for 201612040cin
Found snaps for 201612040rai
Found snaps for 201612050nyj
Found snaps for 201612040nwe
Found snaps for 201612040nor
Found snaps for 201612010min
Found snaps for 201612040rav
Found snaps for 201612040atl
Found snaps for 201612040jax
Found snaps fo

Found snaps for 201511260det
Found snaps for 201511290nyj
Found snaps for 201511290den
Found snaps for 201511290htx
Found snaps for 201511290atl
Found snaps for 201511290kan
Found snaps for 201511260gnb
Found snaps for 201511260dal
Found snaps for 201511300cle
Found snaps for 201511220car
Found snaps for 201511190jax
Found snaps for 201511220phi
Found snaps for 201511220rav
Found snaps for 201511220sea
Found snaps for 201511220sdg
Found snaps for 201511220det
Found snaps for 201511220htx
Found snaps for 201511230nwe
Found snaps for 201511220min
Found snaps for 201511220mia
Found snaps for 201511220atl
Found snaps for 201511220chi
Found snaps for 201511220crd
Found snaps for 201511150was
Found snaps for 201511150oti
Found snaps for 201511150tam
Found snaps for 201511150ram
Found snaps for 201511150sea
Found snaps for 201511150pit
Found snaps for 201511150phi
Found snaps for 201511150rai
Found snaps for 201511120nyj
Found snaps for 201511150nyg
Found snaps for 201511150den
Found snaps fo

Found snaps for 201411090jax
Found snaps for 201411090gnb
Found snaps for 201411060cin
Found snaps for 201411020min
Found snaps for 201411020cle
Found snaps for 201411020sfo
Found snaps for 201411020sea
Found snaps for 201411020mia
Found snaps for 201411020pit
Found snaps for 201411020htx
Found snaps for 201411020kan
Found snaps for 201411030nyg
Found snaps for 201411020nwe
Found snaps for 201410300car
Found snaps for 201411020cin
Found snaps for 201411020dal
Found snaps for 201410270dal
Found snaps for 201410260oti
Found snaps for 201410260tam
Found snaps for 201410260kan
Found snaps for 201410260car
Found snaps for 201410230den
Found snaps for 201410260pit
Found snaps for 201410260crd
Found snaps for 201410260cle
Found snaps for 201410260nyj
Found snaps for 201410260nwe
Found snaps for 201410260nor
Found snaps for 201410260jax
Found snaps for 201410260atl
Found snaps for 201410260cin
Found snaps for 201410190was
Found snaps for 201410190ram
Found snaps for 201410190den
Found snaps fo

/home/welced12/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


Found snaps for 201311030nwe
Found snaps for 201311030rai
Found snaps for 201311030nyj
Found snaps for 201311030dal
Found snaps for 201310310mia
Found snaps for 201311030buf
Found snaps for 201311030htx
Found snaps for 201311040gnb
Found snaps for 201311030cle
Found snaps for 201311030car
Found snaps for 201310270den
Found snaps for 201310240tam
Found snaps for 201310280ram
Found snaps for 201310270jax
Found snaps for 201310270rai
Found snaps for 201310270phi
Found snaps for 201310270cin
Found snaps for 201310270nwe
Found snaps for 201310270nor
Found snaps for 201310270min
Found snaps for 201310270kan
Found snaps for 201310270det
Found snaps for 201310270crd
Found snaps for 201310200was
Found snaps for 201310200oti
Found snaps for 201310200atl
Found snaps for 201310200car
Found snaps for 201310170crd
Found snaps for 201310200jax
Found snaps for 201310200pit
Found snaps for 201310200phi
Found snaps for 201310200nyj
Found snaps for 201310210nyg
Found snaps for 201310200mia
Found snaps fo

Found snaps for 201210210tam
Found snaps for 201210210ram
Found snaps for 201210180sfo
Found snaps for 201210210cin
Found snaps for 201210210rai
Found snaps for 201210210nwe
Found snaps for 201210210min
Found snaps for 201210210clt
Found snaps for 201210210htx
Found snaps for 201210220chi
Found snaps for 201210210car
Found snaps for 201210140was
Found snaps for 201210110oti
Found snaps for 201210140tam
Found snaps for 201210140mia
Found snaps for 201210140sfo
Found snaps for 201210140sea
Found snaps for 201210150sdg
Found snaps for 201210140phi
Found snaps for 201210140atl
Found snaps for 201210140nyj
Found snaps for 201210140htx
Found snaps for 201210140rav
Found snaps for 201210140cle
Found snaps for 201210140crd
Found snaps for 201210070was
Found snaps for 201210070min
Found snaps for 201210040ram
Found snaps for 201210070sfo
Found snaps for 201210070car
Found snaps for 201210070nor
Found snaps for 201210070pit
Found snaps for 201210080nyj
Found snaps for 201210070nyg
Found snaps fo

,away,fumbles,fumbles_lost,gid,home,off_pct,offense,pass_att,pass_cmp,pass_int,...,rush_att,rush_long,rush_td,rush_yds,season,special_teams,st_pct,targets,team,week
12,ram,0,0,201610020crd,crd,21%,16,0,0,0,...,6,18,0,34,2016,0,0%,1,ARI,4
19,ram,0,0,201310130htx,htx,87%,66,0,0,0,...,0,0,0,0,2013,4,16%,4,HOU,6
19,chi,0,0,201312010min,min,44%,38,0,0,0,...,0,0,0,0,2013,5,14%,1,MIN,13
5,cin,0,0,201312010sdg,sdg,39%,24,0,0,0,...,0,0,0,0,2013,2,10%,3,CIN,13
4,kan,0,0,201211010sdg,sdg,10%,6,0,0,0,...,1,6,1,6,2012,13,57%,2,KAN,9


In [49]:
ind_stats_w_snaps_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30455 entries, 0 to 23
Data columns (total 34 columns):
away               30455 non-null object
fumbles            30455 non-null object
fumbles_lost       30455 non-null object
gid                30455 non-null object
home               30455 non-null object
off_pct            30437 non-null object
offense            30437 non-null object
pass_att           30455 non-null int64
pass_cmp           30455 non-null int64
pass_int           30455 non-null int64
pass_long          30455 non-null int64
pass_rating        30455 non-null object
pass_sacked        30455 non-null int64
pass_sacked_yds    30455 non-null int64
pass_td            30455 non-null int64
pass_yds           30455 non-null int64
player             0 non-null object
player_a_x         30455 non-null object
player_href        30455 non-null object
pos                30437 non-null object
rec                30455 non-null int64
rec_long           30455 non-null int64
rec_td

In [50]:
ind_stats_w_snaps_df.drop(columns='player')
ind_stats_w_snaps_df.to_csv("../data/pfr_ind_off_snaps.csv")

In [71]:
sdf = ind_stats_w_snaps_df
sdf.rename(index=str, columns={'player_a_x':'player_a'}, inplace=True)
sdf['offense'] = sdf['offense'].fillna(0).astype(int)
sdf['fumbles'] = sdf['fumbles'].fillna(0).astype(int)
sdf[ sdf.pos.isnull() ][rb_cols]

ValueError: invalid literal for int() with base 10: ''

In [56]:
sdf.player_href.unique()

array(['/players/C/CousKi00.htm', '/players/B/BibbKa00.htm',
       '/players/P/PeriSa00.htm', ..., '/players/H/HeapTo00.htm',
       '/players/B/BossKe00.htm', '/players/T/TurnRo22.htm'], dtype=object)

In [69]:
def lookup(df, pname):
    ndf = df[
        df.player_a == pname
    ]
    return ndf.player_href.unique()
lookup(sdf, 'Cam Newton')

array(['/players/N/NewtCa00.htm'], dtype=object)

# Start looking at year-to-year performance

Potential Inputs:
- Stats from previous seasons
 - Games 1-8
 - Games 8-13
 - Games 13-16
Things to add later:
- Seniority
 - Years pro
 - New team?
- Draft information
 - Pick number
 - Combine results
  - Whether/how to impute missing numbers

In [70]:
# Get a player's stats for a season
def season_stats(df, pid, season):
    pdf = df[
        (df.player_href == pid) &
        (df.season == season)
    ]
    sum_cols = [
        'offense', 'pass_att', 'pass_cmp', 'pass_td', 'pass_int', 
        'rush_att', 'rush_yds', 'rush_td', 'targets', 'rec', 
        'rec_yds', 'rec_td', 'fumbles'
    ]
    
    sums = pdf[sum_cols].agg(['mean','sum'])
    sums['games'] = pdf['offense'].agg('count')
    return sums


season_stats(sdf, '/players/N/NewtCa00.htm', 2017)

,offense,pass_att,pass_cmp,pass_td,pass_int,rush_att,rush_yds,rush_td,targets,rec,rec_yds,rec_td,fumbles,games
mean,65.6875,30.75,18.1875,1.375,1.0,8.6875,47.125,0.375,0.0,0.0,0.0,0.0,1.31251e+14,16
sum,1051.0000,492.00,291.0000,22.000,16.0,139.0000,754.000,6.000,0.0,0.0,0.0,0.0,2100010001010201,16
